In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt


from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('Dataset/winequality-red.csv',sep = ';')

In [3]:
X = df.drop(columns = 'quality') #data matrix
y = df['quality']  #quality

X = np.array(X)  # to np array 
y = np.array(y)   # to np array


In [4]:
from sklearn.model_selection import train_test_split  ## from Assign1
best_features = ['volatile acidity', 'density', 'pH', 'sulphates', 'alcohol']
best_df = df[best_features]
best_df

X_train, X_test, y_train, y_test = train_test_split(best_df, y, test_size=0.2, random_state=42) ## from assignment 1\

In [5]:
from sklearn.preprocessing import StandardScaler  ## from Assignment 1
## 0 mean and unit variance for both X_train and X_test

scaler = StandardScaler()

# Fit on training set only.
scaler.fit(X_train)

# Apply transform to both the training set and the test set.
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
import itertools
import functools 

def polynomialFeatures(x,degree = 2):
        x_t = x.T
        features = np.ones(len(x))
        for degree in range(1, degree + 1):
            for items in itertools.combinations_with_replacement(x_t, degree):
                features = np.vstack((features,functools.reduce(lambda x, y: x * y, items)))
        return features.T

In [7]:
def k_partition(data,kfold):
    return np.array_split(data,kfold)

In [8]:
def learning_curve(model, X, Y, cv, train_size = 1, learning_rate = 0.01, 
                   epochs = 1000, tol = None, regularizer = None, lambd = 0.0, **kwargs):
    
    
    if type(train_size) == int and train_size > 1: 
        train_size_abs = train_size
    elif train_size > 0 and train_size <= 1:
        train_size_abs = int(train_size * X.shape[0])
    else:
        print(f'unaceptable train_size of {train_size}')
        
    if X.shape[0] % train_size_abs != 0:
        t = X.shape[0] // train_size_abs + 1
    else:
        t = X.shape[0] // train_size_abs
    
    t0 = 1
    num_samples_list = []
    rmse_training_list = []
    rmse_validation_list = []

    while t0 <= t:
        i = t0 * train_size_abs
        if i >= X.shape[0]: i = X.shape[0]
        index = np.arange(i)
        X_split = (X[index])
        Y_split = (Y[index])
        
        # using the function_2 to calculate the mse, and then rmse using k-fold based on varying training samples.
        rmse_tr, rmse_va = np.sqrt(kfold_error(model, X_split, Y_split, cv,
                                               learning_rate = learning_rate, epochs = epochs, tol = tol, 
                                               regularizer = regularizer, lambd = lambd))
        num_samples_list.append(i)
        rmse_training_list.append(rmse_tr)
        rmse_validation_list.append(rmse_va)
        t0 += 1

    result = {'num_samples':pd.Series(num_samples_list), 
              'train_scores':pd.Series(rmse_training_list), 'val_score':pd.Series(rmse_validation_list)}
    learning_curve_elements = pd.DataFrame(result)
    
    return np.array(learning_curve_elements["train_scores"]), \
            np.array(learning_curve_elements["val_score"]), np.array(learning_curve_elements["num_samples"]), \
            learning_curve_elements

In [9]:
def kfold_error(model, X, Y, k_fold, learning_rate = 0.01, epochs = 1000, 
                tol = None, regularizer = None, lambd = 0.0, **kwargs):
    if(Y.shape == (Y.shape[0],)):
        Y = np.expand_dims(Y,axis=1)
    dataset = np.concatenate([X,Y],axis=1)

    k_part = k_partition(dataset, k_fold)   # using the function_1 k_partition
    
    error_training = []
    error_validation = []

    for idx,val in enumerate(k_part):
        validation_Y = val[:,-1]
        validation_X = val[:,:-1]
        train = np.concatenate(np.delete(k_part,idx,0))
        train_Y = train[:,-1]
        train_X = train[:,:-1]          
    
        # with sklearn Linearregression to test the entire function
        # replace it when our modeling function is done.
        reg = model().fit(train_X, train_Y)
        pr_train_Y = reg.predict(train_X)
        pr_validation_Y = reg.predict(validation_X)
        mse_train_Y = mse(train_Y, pr_train_Y)
        mse_validation_Y = mse(validation_Y, pr_validation_Y)
    
#         # using our modeling function
#         model.fit(train_X, train_Y, learning_rate = learning_rate, epochs = epochs, 
#                   tol = tol, regularizer = regularizer, lambd = lambd)
#         pr_train_Y = model.predict(train_X)
#         pr_validation_Y = model.predict(validation_X)
#         mse_train_Y = mse(train_Y, pr_train_Y)
#         mse_validation_Y = mse(validation_Y, pr_validation_Y)

        error_training.append(mse_train_Y)
        error_validation.append(mse_validation_Y)

    # return the average mse for the training and the validation fold.
    return np.array(error_training).mean(), np.array(error_validation).mean()

In [10]:
def mse(Y_true, Y_pred):
    E = np.array(Y_true).reshape(-1,1) - np.array(Y_pred).reshape(-1,1)
    mse = 1/np.array(Y_true).shape[0] * (E.T.dot(E))
    return mse[(0,0)]

In [11]:
def plot_polynomial_model_complexity(model, X, Y, cv, maxPolynomialDegree, 
                                     learning_rate=0.01, epochs=1000, tol=None, regularizer=None, lambd=0.0, **kwargs):
    
    poly_list = []
    rmse_training_list = []
    rmse_validation_list = []
    
    for i in range(1, maxPolynomialDegree+1):
        
        # Here should be replaced with our #1 function polynomialFeatures(X, degree)
        X_poly = polynomialFeatures(X, i)
        Y_poly = polynomialFeatures(X, i)
        
        rmse_tr, rmse_va = np.sqrt(kfold_error(model, X_poly, Y_poly, cv, 
                                               learning_rate = learning_rate, epochs = epochs, tol = tol, 
                                               regularizer = regularizer, lambd = lambd))
        poly_list.append(i)
        rmse_training_list.append(rmse_tr)
        rmse_validation_list.append(rmse_va)
        
    # plot the RMSE using the list above.
    plt.figure(figsize = (10, 10))
    
    # linewidth and fontsize
    lw = 2
    fontsize = 20
    
    # plot curves
    plt.plot(poly_list, rmse_training_list, 'o-', color='green', lw = lw, label = "Training set") 
    plt.plot(poly_list, rmse_validation_list, 'o-', color='red', lw = lw, label = "Validation set") 
            
    # add title, xlabel, ylabel, and legend. 
    plt.title('RMSE curve', fontsize = fontsize)
    plt.xlabel('Polynomial Degree', fontsize = fontsize)
    plt.ylabel('RMSE', fontsize = fontsize)
    plt.legend(loc="best", fontsize = fontsize)
    plt.xticks(np.arange(1, i+1, 1))
    
    plt.show()

In [ ]:
X_train = polynomialFeatures(X_train,degree = 3)

In [ ]:
X_train.shape

In [ ]:
from sklearn.linear_model import LinearRegression
cv = 5
poly = 5
#plot_polynomial_model_complexity(LinearRegression, X_train, Y_train, cv, poly, learning_rate = 0.01, 
#                   epochs = 500, tol = None, regularizer = l2, lambd = 0.0)
plot_polynomial_model_complexity(LinearRegression, X_train, y_train, cv, poly, learning_rate = 0.01, 
                   epochs = 500, tol = None, regularizer = None, lambd = 0.0)

In [ ]:

X_train = polynomialFeatures(X_train,5)


In [ ]:
X_train.shape[1]

In [ ]:
a = np.array([[1,2],[3,4]])
a = np.c_[np.ones((a.shape[0],1)),a]
a

In [12]:
class SGD:
    def __init__(self):
        pass   
    
    def fit(self, X, Y, learning_rate=0.01, epochs=1000, tol=None, regularizer=None,lambd=0.0,**kwargs):
        self.degree = 1
        if (kwargs != None):
            self.degree = kwargs['degree']
        self.X = polynomialFeatures(X,self.degree)
        epch_counter = 0
        m,n = self.X.shape
        self.theta = np.zeros(n) # Initialize theta to be 0 of size n
        self.error = 1e10
        while epch_counter < epochs:
            epch_counter += 1 #count the epochs
            for i in range(m):
                theta = self.theta
                error = self.error
                a = np.dot(self.X[i].T , (self.X[i] @ self.theta - Y[i]))
                if(regularizer == 'l2'):
                #j_theta = j_theta + (np.dot(theta[1:].T,theta[1:]) *(lambd /(2*m)))
                    self.theta = theta - (a * learning_rate) - (learning_rate * lambd*theta)
                
                elif(regularizer == 'l1'):
                #j_theta = j_theta + (theta[1:].sum() *(lambd /(2*m)))
                    self.theta = theta - (a * learning_rate) - (learning_rate * lambd* np.sign(theta))
                else:
                    self.theta = theta - (a * learning_rate)
                if np.linalg.norm(self.theta - theta, ord=1) < tol:
                    break
        print(f'Epochs needed: {epch_counter}')
                                  
    def predict(self,X):
        X = polynomialFeatures(X,self.degree)
        return X @ self.theta

In [13]:
class Linear_Regression:
    def __init__(self):
        pass   
    
    def fit(self, X, Y, learning_rate=0.01, epochs=1000, tol=None, regularizer=None,lambd=0.0,**kwargs):
        self.degree = 1
        if (kwargs != None):
            self.degree = kwargs['degree']
        self.X = polynomialFeatures(X,self.degree)
        epch_counter = 0
        m,n = self.X.shape
        self.theta = np.zeros(n) # Initialize theta to be 0 of size n
        self.error = 1e10
        self.j_theta = 1e10
        while epch_counter < epochs:
            epch_counter += 1 #count the epochs
            theta = self.theta
            error = self.error
            j_theta = self.j_theta
            self.j_theta = (((Y- self.X @ theta) ** 2).mean()) / 2
            print(f'Epoch{epch_counter}: j_theta={self.j_theta}')
            a = np.dot(self.X.T,(self.X @ theta - Y)) # gradient of loss wrt parameter
            print(f'Epoch{epch_counter}: j_theta={a}')
            if(regularizer == 'l2'):
                self.j_theta = self.j_theta + (np.dot(theta[1:].T,theta[1:]) *(lambd /(2*m)))
                self.theta = theta - ((a * learning_rate) / m) - (learning_rate * lambd*theta)/m 
                print(f'\n\nEpoch{epch_counter}: theta={self.theta}')
                
            elif(regularizer == 'l1'):
                self.j_theta = selfj_theta + (theta[1:].sum() *(lambd /(2*m)))
                self.theta = theta - ((a * learning_rate) / m) - (learning_rate * lambd* np.sign(theta))/m
            else:
                self.theta = theta - ((a * learning_rate) / m)
                
            #self.error = mean_squared_error(Y, self.X @ self.theta)
            #if np.abs(self.error - error) < tol:
                #break
            if np.abs(self.j_theta - j_theta) < tol:
                break
        print(f'Epochs needed: {epch_counter}')
                                  
    def predict(self,X):
        X = polynomialFeatures(X,self.degree)
        return X @ self.theta

In [14]:
lin_reg_sgd = Linear_Regression()

In [15]:
lin_reg_sgd.fit(X_train,y_train,learning_rate=0.01,epochs=3,regularizer='l2',lambd= 0.1, tol=10,degree=2)

Epoch1: j_theta=16.139562157935888
Epoch1: j_theta=[-7193.           390.34094757   172.37659605    46.61401311
  -250.27039341  -487.62830155 -6985.16937956  -332.07921876
 -1570.16277466  1905.21705836  1650.45018469 -7436.00527532
  2457.18557757  -941.75400489  3673.77015313 -7180.27064955
  1437.80749234 -1526.60694007 -7035.30428062  -792.51437935
 -7554.35093361]


Epoch1: theta=[ 0.05623925 -0.00305192 -0.00134775 -0.00036446  0.00195677  0.00381257
  0.0546143   0.0025964   0.01227649 -0.01489615 -0.01290422  0.05813921
 -0.01921177  0.00736321 -0.02872377  0.05613972 -0.01124165  0.01193594
  0.05500629  0.00619636  0.05906451]
Epoch2: j_theta=14.148466475748146
Epoch2: j_theta=[-6712.87942648   413.35288688   102.61778629    23.37523425
   -29.91380413  -340.14582557 -6229.72783987  -296.13651422
 -1353.48766669  1688.01669633  1513.00263442 -6527.61330831
  2038.52373097  -748.83479985  3127.57903256 -6211.3561668
   868.6363014  -1174.80892297 -5343.57890925  -899.9221793


In [16]:
# Make prediction 
from sklearn.metrics import mean_squared_error
y_train_predicted_sgd = lin_reg_sgd.predict(X_train)


#lin_reg_sgd.fit(X_test,y_test)
y_test_predicted_sgd = lin_reg_sgd.predict(X_test)
print("Training: Mean squared error: %.2f"
      % mean_squared_error(y_train, y_train_predicted_sgd))

print("Test: Mean squared error: %.2f"
      % mean_squared_error(y_test, y_test_predicted_sgd))

Training: Mean squared error: 25.27
Test: Mean squared error: 25.50


In [17]:
class Linear_Regression:
    def __init__(self):
        pass   
    
    def fit(self, X, Y, learning_rate=0.01, epochs=1000, tol=None, regularizer=None,lambd=0.0,**kwargs):
        epch_counter = 0
        m,n = X.shape
        self.theta = np.zeros(n) # Initialize theta to be 0 of size n
        self.error = -tol
        while epch_counter < epochs:
            epch_counter += 1 #count the epochs
            theta = self.theta
            error = self.error
            #j_theta = (((Y- X @ theta) ** 2).mean()) / 2
            a = np.dot(X.T,(X @ theta - Y)) # gradient of loss wrt parameter
            
            if(regularizer == 'l2'):
                #j_theta = j_theta + (np.dot(theta[1:].T,theta[1:]) *(lambd /(2*m)))
                self.theta = theta - ((a * learning_rate) / m) - (learning_rate * lambd*theta)/m
                print(self.theta)
            elif(regularizer == 'l1'):
                #j_theta = j_theta + (theta[1:].sum() *(lambd /(2*m)))
                self.theta = theta - ((a * learning_rate) / m) - (learning_rate * lambd* np.sign(theta))/m
            else:
                self.theta = theta - ((a * learning_rate) / m)
            
            self.error = mean_squared_error(Y,X @ self.theta)
            
            if np.abs(self.error - error) < tol:
                break

        print(f'Epochs needed: {epch_counter}')
                                  
    def predict(self,X):
        return X @ self.theta

In [18]:
a = np.array([1,2,3])
np.dot(a,2)

array([2, 4, 6])

In [20]:
class SGD:
    def __init__(self):
        pass   
    
    def fit(self, X, Y, learning_rate=0.01, epochs=1000, tol=None, regularizer=None,lambd=0.0,**kwargs):
        epch_counter = 0
        m,n = X.shape
        self.theta = np.zeros(n) # Initialize theta to be 0 of size n
        self.error = -tol
        while epch_counter < epochs:
            epch_counter += 1 #count the epochs
            for i in range(m):
                theta = self.theta
                error = self.error
                a = np.dot(X[i].T , (X[i] @ self.theta - Y[i]))
                if(regularizer == 'l2'):
                #j_theta = j_theta + (np.dot(theta[1:].T,theta[1:]) *(lambd /(2*m)))
                    self.theta = theta - (a * learning_rate) - (learning_rate * lambd*theta)
                    print(self.theta)
                
                elif(regularizer == 'l1'):
                #j_theta = j_theta + (theta[1:].sum() *(lambd /(2*m)))
                    self.theta = theta - (a * learning_rate) - (learning_rate * lambd* np.sign(theta))
                else:
                    self.theta = theta - (a * learning_rate)
                #if np.linalg.norm(self.theta - theta, ord=1) < tol:
                #    break
                self.error = mean_squared_error(Y,X @ self.theta)
            
                if np.abs(self.error - error) < tol:
                    break
        print(f'Epochs needed: {epch_counter}')
                                  
    def predict(self,X):
        return X @ self.theta

In [21]:
lin_reg_sgd = Linear_Regression()

In [22]:
X_train = polynomialFeatures(X_train,2)
X_test = polynomialFeatures(X_test,2)
lin_reg_sgd.fit(X_train,y_train,learning_rate=0.01,epochs=500,regularizer='l2',lambd= 1e5, tol=1e-5)

[ 0.05623925 -0.00305192 -0.00134775 -0.00036446  0.00195677  0.00381257
  0.0546143   0.0025964   0.01227649 -0.01489615 -0.01290422  0.05813921
 -0.01921177  0.00736321 -0.02872377  0.05613972 -0.01124165  0.01193594
  0.05500629  0.00619636  0.05906451]
[ 0.06475336 -0.00389759 -0.00109632 -0.00026226  0.00066073  0.00349114
  0.06062132  0.00288175  0.01326037 -0.01644737 -0.01464449  0.06371929
 -0.02012926  0.00746105 -0.03071909  0.06081043 -0.00924377  0.01178907
  0.05377838  0.00838781  0.06549447]
[ 0.06629049 -0.00410754 -0.00098468 -0.00029181  0.00043513  0.00329679
  0.06138481  0.00289713  0.01333416 -0.01666158 -0.014905    0.06430629
 -0.0200863   0.00743767 -0.03077958  0.06133204 -0.00885247  0.01158802
  0.05348083  0.00878536  0.06631855]
[ 0.06658592 -0.00415611 -0.00095577 -0.00030436  0.00039931  0.0032423
  0.0614842   0.00289421  0.01333454 -0.01669343 -0.01494578  0.0643646
 -0.02005448  0.00743045 -0.03075397  0.06139566 -0.00878652  0.01153175
  0.05344559

In [23]:
# Make prediction 
from sklearn.metrics import mean_squared_error

y_train_predicted_sgd = lin_reg_sgd.predict(X_train)


#lin_reg_sgd.fit(X_test,y_test)
y_test_predicted_sgd = lin_reg_sgd.predict(X_test)
print("Training: Mean squared error: %.2f"
      % mean_squared_error(y_train, y_train_predicted_sgd))

print("Test: Mean squared error: %.2f"
      % mean_squared_error(y_test, y_test_predicted_sgd))

Training: Mean squared error: 27.92
Test: Mean squared error: 28.35
